## 크롤링 코드

In [ ]:
!pip install selenium pandas webdriver_manager

In [ ]:
# # Colab 환경일 경우
# !apt-get update
# !apt-get install -y chromium-chromedriver
# !pip install selenium webdriver-manager requests pandas

### 전체

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import requests
import pandas as pd
import time
import os

############################# 드라이버 설정 #############################

# chrome options
options = Options()
options.add_argument("--headless") # 백그라운드로 실행

# webdriver 설정
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service, options=options)

############################ csv 파일 초기화 ############################

# 잎을 관상하는 실내식물
leaf_info_columns = ["이름", "학명", "영명", "유통명", "과명", "원산지", "TIP"]
leaf_detail_columns = ["이름", "분류", "생육형태", "생장높이", "생장너비", "실내정원구성", 
                       "생태형", "잎형태", "잎무늬", "잎색", "꽃피는 계절", "꽃색", 
                       "열매 맺는 계절", "열매색", "향기", "번식방법", "번식시기"]
leaf_manage_columns = ["이름", "관리수준", "관리요구도", "광요구도", "배치장소",
                       "생장속도", "생육적온", "겨울최저온도", "습도", "비료", "토양", 
                       "물주기-봄", "물주기-여름", "물주기-가을", "물주기-겨울", "병충해"]

leaf_info_df = pd.DataFrame(columns=leaf_info_columns)
leaf_detail_df = pd.DataFrame(columns=leaf_detail_columns)
leaf_manage_df = pd.DataFrame(columns=leaf_manage_columns)

# 건조에 강한 실내식물
dry_info_columns = ["이름", "학명", "유통명", "형태분류", "원산지", "꽃", "엽색변화", "뿌리형태", 
                    "생장형", "생장속도", "특성", "월동온도", "생육온도"]
dry_manage_columns = ["이름", "광", "물주기", "번식", "관리수준", "관리요구도", "배치장소", "병충해", "TIP"]

dry_info_df = pd.DataFrame(columns=dry_info_columns)
dry_manage_df = pd.DataFrame(columns=dry_manage_columns)

# 분류
list_columns = ["name", "class"]
list_df = pd.DataFrame(columns=list_columns)

############################# 크롤링 공통 #############################

def get_text_or_empty(xpath):
    """
    요소에서 텍스트를 추출하거나 요소가 없을 경우 빈 문자열 반환
    """
    try:
        return driver.find_element(By.XPATH, xpath).text.strip()
    except:
        return ""

def get_info_dict(label_xpath, value_xpath):
    """
    크롤링 데이터를 딕셔너리 형태로 반환 (끝날 때까지 자동 탐색)
    """
    info_dict = {}
    i = 1  # 인덱스 초기화

    while True:
        try:
            column_name = get_text_or_empty(label_xpath.format(i=i))
            column_value = get_text_or_empty(value_xpath.format(i=i))

            if not column_name and not column_value:
                break

            info_dict[column_name] = column_value
            i += 1
        except:
            break
    return info_dict

def download_img(name, xpath):
    """
    지정된 xpath의 이미지를 name.jpg로 저장
    """
    try:
        img_url = driver.find_element(By.XPATH, xpath).get_attribute('src')
        os.makedirs('./img', exist_ok=True)

        if img_url:
            response = requests.get(img_url)
            if response.status_code == 200:
                file_path = f'./img/{name}.jpg'
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"이미지 저장 완료: {file_path}")
    
    except:
        print(f"이미지 저장 실패: {name}.jpg")
        pass

############################# leaf 크롤링 #############################

def get_leaf_info():
    """
    식물별 상세 정보 크롤링
    """
    name = get_text_or_empty('//*[@id="contentForm"]/div[2]/div/div/dl/dt/strong')
    download_img(name, '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[1]/div/dl/dd/img')

    # 기본 정보 - leaf_info.csv
    info_values = get_info_dict(
        label_xpath = '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[2]/ul/li[{i}]/label',
        value_xpath = '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[2]/ul/li[{i}]/span'
    )
    info_values['이름'] = name
    leaf_info_df.loc[len(leaf_info_df)] = info_values

    # 상세 정보 - leaf_detail.csv
    detail_values = get_info_dict(
        label_xpath = '//*[@id="contentForm"]/div[4]/table/tbody/tr[{i}]/th',
        value_xpath = '//*[@id="contentForm"]/div[4]/table/tbody/tr[{i}]/td'
    )
    detail_values['이름'] = name
    leaf_detail_df.loc[len(leaf_detail_df)] = detail_values

    # 관리 정보 - leaf_manage.csv
    manage_values = get_info_dict(
        label_xpath = '//*[@id="contentForm"]/div[5]/table/tbody/tr[{i}]/th',
        value_xpath = '//*[@id="contentForm"]/div[5]/table/tbody/tr[{i}]/td'
    )
    manage_values['이름'] = name
    leaf_manage_df.loc[len(leaf_manage_df)] = manage_values

    # 리스트 정보 - list.csv
    list_df.loc[len(list_df)] = {"name": name, "class": "leaf"}

def get_leaf_page(base_url, start, end):
    """
    각 페이지를 방문하여 잎을 관상하는 실내식물 정보 크롤링
    """
    print("잎을 관상하는 실내식물 크롤링 시작:")
    for p in range(start, end+1):
        try:
            # 페이지 이동
            if p == 1:
                driver.get(base_url)
            elif p == 11:
                driver.get(base_url)
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[1]'
                driver.find_element(By.XPATH, next_xpath).click()
            elif p == 21:
                driver.get(base_url)
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[1]'
                driver.find_element(By.XPATH, next_xpath).click()
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[3]'
                driver.find_element(By.XPATH, next_xpath).click()
            elif p%10 == 0:
                next_xpath = '//*[@id="gardenPlant"]/div[3]/span[10]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            else:
                next_xpath = f'//*[@id="gardenPlant"]/div[3]/span[{p%10}]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            time.sleep(2)
            
            # 식물 정보 크롤링
            for plant in range(1, 9):
                try:
                    # 식물 페이지로 이동
                    if p == 10 and plant > 6:
                        plant_xpath = f'//*[@id="gardenPlant"]/div[2]/ul/i/i/li[{plant-5}]/a'
                    else:
                        plant_xpath = f'//*[@id="gardenPlant"]/div[2]/ul/li[{plant}]/a'
                    driver.find_element(By.XPATH, plant_xpath).click()
                    time.sleep(2)

                    # 식물 정보 크롤링
                    get_leaf_info()

                    # 이전 페이지로 이동
                    driver.back()
                    time.sleep(2)
                
                except Exception as e:
                    print(f"Error processing plant {plant}: {e}")
                    continue

        except Exception as e:
            print(f"Error processing page {p}: {e}")
            continue

############################## dry 크롤링 ##############################

def get_dry_info():
    """
    식물별 상세 정보 크롤링
    """
    name = get_text_or_empty('//*[@id="contentForm"]/div[2]/div/div/dl/dt/strong')
    download_img(name, '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[1]/div/img')

    # 기본 정보 - dry_info.csv
    info_values = get_info_dict(
        label_xpath = '//*[@id="sedumInfo"]/div/div[1]/ul/li[{i}]/strong',
        value_xpath = '//*[@id="sedumInfo"]/div/div[1]/ul/li[{i}]/span'
    )
    info_values['이름'] = name
    dry_info_df.loc[len(dry_info_df)] = info_values

    # 관리 정보 - manage.csv
    manage_values = get_info_dict(
        label_xpath = '//*[@id="sedumInfo"]/div/div[2]/ul/li[{i}]/strong',
        value_xpath = '//*[@id="sedumInfo"]/div/div[2]/ul/li[{i}]/span'
    )
    manage_values['이름'] = name
    dry_manage_df.loc[len(dry_manage_df)] = manage_values

    # 리스트 정보 - list.csv
    list_df.loc[len(list_df)] = {"name": name, "class": "dry"}

def get_dry_page(base_url, start, end):
    """
    각 페이지를 방문하여 ㅓㄱㄴ조에 강한 실내식물 정보 크롤링
    """
    print("건조에 강한 실내식물 크롤링 시작:")
    for p in range(start, end+1):
        try:
            # 페이지 이동
            if p == 1:
                driver.get(base_url)
            elif p == 11:
                driver.get(base_url)
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[1]'
                driver.find_element(By.XPATH, next_xpath).click()
            elif p%10 == 0:
                next_xpath = '//*[@id="gardenPlant"]/div[3]/span[10]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            else:
                next_xpath = f'//*[@id="gardenPlant"]/div[3]/span[{p%10}]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            time.sleep(2)
            
            # 식물 정보 크롤링
            for plant in range(1, 9):
                try:
                    # 식물 페이지로 이동
                    plant_xpath = f'//*[@id="gardenPlant"]/div[2]/ul/li[{plant}]/a'
                    driver.find_element(By.XPATH, plant_xpath).click()
                    time.sleep(2)

                    # 식물 정보 크롤링
                    get_dry_info()

                    # 이전 페이지로 이동
                    driver.back()
                    time.sleep(2)
                
                except Exception as e:
                    print(f"Error processing plant {plant}: {e}")
                    continue

        except Exception as e:
            print(f"Error processing page {p}: {e}")
            continue

############################# csv 파일 저장 #############################

def save_csv():
    """
    크롤링한 정보를 csv 파일로 저장
    """
    # 잎을 관상하는 실내식물
    leaf_info_df.to_csv("leaf_info.csv", index=False, encoding='utf-8-sig')
    leaf_detail_df.to_csv("leaf_detail.csv", index=False, encoding='utf-8-sig')
    leaf_manage_df.to_csv("leaf_manage.csv", index=False, encoding='utf-8-sig')

    # 건조에 강한 실내식물
    dry_info_df.to_csv("dry_info.csv", index=False, encoding='utf-8-sig')
    dry_manage_df.to_csv("dry_manage.csv", index=False, encoding='utf-8-sig')

    # 분류
    list_df.to_csv("list_2.csv", index=False, encoding='utf-8-sig')

############################### 실행 ###############################

# 잎을 관상하는 실내 식물
leaf_url = 'https://www.nongsaro.go.kr/portal/ps/psz/psza/contentMain.ps?menuId=PS00376&pageUnit=8'

# 건조에 강한 실내식물
dry_url = 'https://www.nongsaro.go.kr/portal/ps/psz/psza/contentMain.ps?menuId=PS04099&pageUnit=8'

get_leaf_page(leaf_url, 1, 28)
get_dry_page(dry_url, 1, 13)

save_csv()
driver.quit()
print('크롤링 완료!')

### 잎을 관상하는 실내식물

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import requests
import pandas as pd
import time
import os

############################# 드라이버 설정 #############################

# chrome options
options = Options()
options.add_argument("--headless") # 백그라운드로 실행

# webdriver 설정
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service, options=options)

############################ csv 파일 초기화 ############################

# 잎을 관상하는 실내식물
leaf_info_columns = ["이름", "학명", "영명", "유통명", "과명", "원산지", "TIP"]
leaf_detail_columns = ["이름", "분류", "생육형태", "생장높이", "생장너비", "실내정원구성", 
                       "생태형", "잎형태", "잎무늬", "잎색", "꽃피는 계절", "꽃색", 
                       "열매 맺는 계절", "열매색", "향기", "번식방법", "번식시기"]
leaf_manage_columns = ["이름", "관리수준", "관리요구도", "광요구도", "배치장소",
                       "생장속도", "생육적온", "겨울최저온도", "습도", "비료", "토양", 
                       "물주기-봄", "물주기-여름", "물주기-가을", "물주기-겨울", "병충해"]

leaf_info_df = pd.DataFrame(columns=leaf_info_columns)
leaf_detail_df = pd.DataFrame(columns=leaf_detail_columns)
leaf_manage_df = pd.DataFrame(columns=leaf_manage_columns)

# 분류
list_columns = ["name", "class"]
list_df = pd.DataFrame(columns=list_columns)

############################# 크롤링 공통 #############################

def get_text_or_empty(xpath):
    """
    요소에서 텍스트를 추출하거나 요소가 없을 경우 빈 문자열 반환
    """
    try:
        return driver.find_element(By.XPATH, xpath).text.strip()
    except:
        return ""

def get_info_dict(label_xpath, value_xpath):
    """
    크롤링 데이터를 딕셔너리 형태로 반환 (끝날 때까지 자동 탐색)
    """
    info_dict = {}
    i = 1  # 인덱스 초기화

    while True:
        try:
            column_name = get_text_or_empty(label_xpath.format(i=i))
            column_value = get_text_or_empty(value_xpath.format(i=i))

            if not column_name and not column_value:
                break

            info_dict[column_name] = column_value
            i += 1
        except:
            break
    return info_dict

def download_img(name, xpath):
    """
    지정된 xpath의 이미지를 name.jpg로 저장
    """
    try:
        img_url = driver.find_element(By.XPATH, xpath).get_attribute('src')
        os.makedirs('./img', exist_ok=True)

        if img_url:
            response = requests.get(img_url)
            if response.status_code == 200:
                file_path = f'./img/{name}.jpg'
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"이미지 저장 완료: {file_path}")
    
    except:
        print(f"이미지 저장 실패: {name}.jpg")
        pass

############################# leaf 크롤링 #############################

def get_leaf_info():
    """
    식물별 상세 정보 크롤링
    """
    name = get_text_or_empty('//*[@id="contentForm"]/div[2]/div/div/dl/dt/strong')
    download_img(name, '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[1]/div/dl/dd/img')

    # 기본 정보 - leaf_info.csv
    info_values = get_info_dict(
        label_xpath = '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[2]/ul/li[{i}]/label',
        value_xpath = '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[2]/ul/li[{i}]/span'
    )
    info_values['이름'] = name
    leaf_info_df.loc[len(leaf_info_df)] = info_values

    # 상세 정보 - leaf_detail.csv
    detail_values = get_info_dict(
        label_xpath = '//*[@id="contentForm"]/div[4]/table/tbody/tr[{i}]/th',
        value_xpath = '//*[@id="contentForm"]/div[4]/table/tbody/tr[{i}]/td'
    )
    detail_values['이름'] = name
    leaf_detail_df.loc[len(leaf_detail_df)] = detail_values

    # 관리 정보 - leaf_manage.csv
    manage_values = get_info_dict(
        label_xpath = '//*[@id="contentForm"]/div[5]/table/tbody/tr[{i}]/th',
        value_xpath = '//*[@id="contentForm"]/div[5]/table/tbody/tr[{i}]/td'
    )
    manage_values['이름'] = name
    leaf_manage_df.loc[len(leaf_manage_df)] = manage_values

    # 리스트 정보 - list.csv
    list_df.loc[len(list_df)] = {"name": name, "class": "leaf"}

def get_leaf_page(base_url, start, end):
    """
    각 페이지를 방문하여 잎을 관상하는 실내식물 정보 크롤링
    """
    print("잎을 관상하는 실내식물 크롤링 시작:")
    for p in range(start, end+1):
        try:
            # 페이지 이동
            if p == 1:
                driver.get(base_url)
            elif p == 11:
                driver.get(base_url)
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[1]'
                driver.find_element(By.XPATH, next_xpath).click()
            elif p == 21:
                driver.get(base_url)
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[1]'
                driver.find_element(By.XPATH, next_xpath).click()
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[3]'
                driver.find_element(By.XPATH, next_xpath).click()
            elif p%10 == 0:
                next_xpath = '//*[@id="gardenPlant"]/div[3]/span[10]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            else:
                next_xpath = f'//*[@id="gardenPlant"]/div[3]/span[{p%10}]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            time.sleep(2)
            
            # 식물 정보 크롤링
            for plant in range(1, 9):
                try:
                    # 식물 페이지로 이동
                    if p == 10 and plant > 6:
                        plant_xpath = f'//*[@id="gardenPlant"]/div[2]/ul/i/i/li[{plant-5}]/a'
                    else:
                        plant_xpath = f'//*[@id="gardenPlant"]/div[2]/ul/li[{plant}]/a'
                    driver.find_element(By.XPATH, plant_xpath).click()
                    time.sleep(2)

                    # 식물 정보 크롤링
                    get_leaf_info()

                    # 이전 페이지로 이동
                    driver.back()
                    time.sleep(2)
                
                except Exception as e:
                    print(f"Error processing plant {plant}: {e}")
                    continue

        except Exception as e:
            print(f"Error processing page {p}: {e}")
            continue

############################# csv 파일 저장 #############################

def save_csv():
    """
    크롤링한 정보를 csv 파일로 저장
    """
    # 잎을 관상하는 실내식물
    leaf_info_df.to_csv("leaf_info.csv", index=False, encoding='utf-8-sig')
    leaf_detail_df.to_csv("leaf_detail.csv", index=False, encoding='utf-8-sig')
    leaf_manage_df.to_csv("leaf_manage.csv", index=False, encoding='utf-8-sig')

    # 분류
    list_df.to_csv("list.csv", index=False, encoding='utf-8-sig')

############################### 실행 ###############################

# 잎을 관상하는 실내 식물
leaf_url = 'https://www.nongsaro.go.kr/portal/ps/psz/psza/contentMain.ps?menuId=PS00376&pageUnit=8'

get_leaf_page(leaf_url, 1, 28)

save_csv()
driver.quit()
print('크롤링 완료!')

잎을 관상하는 실내식물 크롤링 시작:
이미지 저장 완료: ./img/가울테리아.jpg
이미지 저장 완료: ./img/개운죽.jpg
이미지 저장 완료: ./img/골드크레스트 '윌마'.jpg
이미지 저장 완료: ./img/공작야자.jpg
이미지 저장 완료: ./img/관엽베고니아.jpg
이미지 저장 완료: ./img/관음죽.jpg
이미지 저장 완료: ./img/구문초.jpg
이미지 저장 완료: ./img/구즈마니아.jpg
이미지 저장 완료: ./img/군자란.jpg
이미지 저장 완료: ./img/글레코마.jpg
이미지 저장 완료: ./img/금목서.jpg
이미지 저장 완료: ./img/금사철나무.jpg
이미지 저장 완료: ./img/금식나무.jpg
이미지 저장 완료: ./img/금전수.jpg
이미지 저장 완료: ./img/금천죽.jpg
이미지 저장 완료: ./img/기누라.jpg
이미지 저장 완료: ./img/꽃베고니아.jpg
이미지 저장 완료: ./img/나도풍란.jpg
이미지 저장 완료: ./img/나한송.jpg
이미지 저장 완료: ./img/남천.jpg
이미지 저장 완료: ./img/네마탄투스.jpg
이미지 저장 완료: ./img/네오레겔리아.jpg
이미지 저장 완료: ./img/녹영.jpg
이미지 저장 완료: ./img/뉴기니아봉선화.jpg
이미지 저장 완료: ./img/대만고무나무.jpg
이미지 저장 완료: ./img/더피고사리.jpg
이미지 저장 완료: ./img/덕구리난.jpg
이미지 저장 완료: ./img/데코라고무나무.jpg
이미지 저장 완료: ./img/덴파레.jpg
이미지 저장 완료: ./img/도깨비고비.jpg
이미지 저장 완료: ./img/돈나무.jpg
이미지 저장 완료: ./img/동백.jpg
이미지 저장 완료: ./img/둥근잎 아랄리아.jpg
이미지 저장 완료: ./img/듀란타.jpg
이미지 저장 완료: ./img/드라세나 '송오브자마이카'.jpg
이미지 저장 완료: ./img/드라세나 '와네끼'.jpg
이미지 저장 완료: ./im

### 건조에 강한 실내식물

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import requests
import pandas as pd
import time
import os

############################# 드라이버 설정 #############################

# chrome options
options = Options()
options.add_argument("--headless") # 백그라운드로 실행

# webdriver 설정
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service, options=options)

############################ csv 파일 초기화 ############################

# 건조에 강한 실내식물
dry_info_columns = ["이름", "학명", "유통명", "형태분류", "원산지", "꽃", "엽색변화", "뿌리형태", 
                    "생장형", "생장속도", "특성", "월동온도", "생육온도"]
dry_manage_columns = ["이름", "광", "물주기", "번식", "관리수준", "관리요구도", "배치장소", "병충해", "TIP"]

dry_info_df = pd.DataFrame(columns=dry_info_columns)
dry_manage_df = pd.DataFrame(columns=dry_manage_columns)

# 분류
list_columns = ["name", "class"]
list_df = pd.DataFrame(columns=list_columns)

############################# 크롤링 공통 #############################

def get_text_or_empty(xpath):
    """
    요소에서 텍스트를 추출하거나 요소가 없을 경우 빈 문자열 반환
    """
    try:
        return driver.find_element(By.XPATH, xpath).text.strip()
    except:
        return ""

def get_info_dict(label_xpath, value_xpath):
    """
    크롤링 데이터를 딕셔너리 형태로 반환 (끝날 때까지 자동 탐색)
    """
    info_dict = {}
    i = 1  # 인덱스 초기화

    while True:
        try:
            column_name = get_text_or_empty(label_xpath.format(i=i)).replace(" ", "")
            column_value = get_text_or_empty(value_xpath.format(i=i))

            if not column_name and not column_value:
                break

            info_dict[column_name] = column_value
            i += 1
        except:
            break
    return info_dict

def download_img(name, xpath):
    """
    지정된 xpath의 이미지를 name.jpg로 저장
    """
    try:
        img_url = driver.find_element(By.XPATH, xpath).get_attribute('src')
        os.makedirs('./img', exist_ok=True)

        if img_url:
            response = requests.get(img_url)
            if response.status_code == 200:
                file_path = f'./img/{name}.jpg'
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"이미지 저장 완료: {file_path}")
    
    except:
        print(f"이미지 저장 실패: {name}.jpg")
        pass

############################## dry 크롤링 ##############################

def get_dry_info():
    """
    식물별 상세 정보 크롤링
    """
    name = get_text_or_empty('//*[@id="contentForm"]/div[2]/div/div/dl/dt/strong')
    download_img(name, '//*[@id="contentForm"]/div[2]/div/div/dl/dd/div[1]/div/img')

    # 기본 정보 - dry_info.csv
    info_values = get_info_dict(
        label_xpath = '//*[@id="sedumInfo"]/div/div[1]/ul/li[{i}]/strong',
        value_xpath = '//*[@id="sedumInfo"]/div/div[1]/ul/li[{i}]/span'
    )
    info_values['이름'] = name
    dry_info_df.loc[len(dry_info_df)] = info_values

    # 관리 정보 - manage.csv
    manage_values = get_info_dict(
        label_xpath = '//*[@id="sedumInfo"]/div/div[2]/ul/li[{i}]/strong',
        value_xpath = '//*[@id="sedumInfo"]/div/div[2]/ul/li[{i}]/span'
    )
    manage_values['이름'] = name
    dry_manage_df.loc[len(dry_manage_df)] = manage_values

    # 리스트 정보 - list.csv
    list_df.loc[len(list_df)] = {"name": name, "class": "dry"}

def get_dry_page(base_url, start, end):
    """
    각 페이지를 방문하여 ㅓㄱㄴ조에 강한 실내식물 정보 크롤링
    """
    print("건조에 강한 실내식물 크롤링 시작:")
    for p in range(start, end+1):
        try:
            # 페이지 이동
            if p == 1:
                driver.get(base_url)
            elif p == 11:
                driver.get(base_url)
                next_xpath = '//*[@id="gardenPlant"]/div[3]/a[1]'
                driver.find_element(By.XPATH, next_xpath).click()
            elif p%10 == 0:
                next_xpath = '//*[@id="gardenPlant"]/div[3]/span[10]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            else:
                next_xpath = f'//*[@id="gardenPlant"]/div[3]/span[{p%10}]/a'
                driver.find_element(By.XPATH, next_xpath).click()
            time.sleep(2)
            
            # 식물 정보 크롤링
            for plant in range(1, 9):
                try:
                    # 식물 페이지로 이동
                    plant_xpath = f'//*[@id="gardenPlant"]/div[2]/ul/li[{plant}]/a'
                    driver.find_element(By.XPATH, plant_xpath).click()
                    time.sleep(2)

                    # 식물 정보 크롤링
                    get_dry_info()

                    # 이전 페이지로 이동
                    driver.back()
                    time.sleep(2)
                
                except Exception as e:
                    print(f"Error processing plant {plant}: {e}")
                    continue

        except Exception as e:
            print(f"Error processing page {p}: {e}")
            continue

############################# csv 파일 저장 #############################

def save_csv():
    """
    크롤링한 정보를 csv 파일로 저장
    """
    # 건조에 강한 실내식물
    dry_info_df.to_csv("dry_info.csv", index=False, encoding='utf-8-sig')
    dry_manage_df.to_csv("dry_manage.csv", index=False, encoding='utf-8-sig')

    # 분류
    list_df.to_csv("list_2.csv", index=False, encoding='utf-8-sig')

############################### 실행 ###############################

# 건조에 강한 실내식물
dry_url = 'https://www.nongsaro.go.kr/portal/ps/psz/psza/contentMain.ps?menuId=PS04099&pageUnit=8'

get_dry_page(dry_url, 1, 13)

save_csv()
driver.quit()
print('크롤링 완료!')

건조에 강한 실내식물 크롤링 시작:
이미지 저장 완료: ./img/페페로미아 그라베올렌스.jpg
이미지 저장 완료: ./img/산세베리아 ‘하니’.jpg
이미지 저장 완료: ./img/미파.jpg
이미지 저장 완료: ./img/벽어연.jpg
이미지 저장 완료: ./img/입전봉.jpg
이미지 저장 완료: ./img/사마로.jpg
이미지 저장 완료: ./img/모난데스 폴리필라.jpg
이미지 저장 완료: ./img/야로수.jpg
이미지 저장 완료: ./img/수.jpg
이미지 저장 완료: ./img/십이지권.jpg
이미지 저장 완료: ./img/알로에.jpg
이미지 저장 완료: ./img/불야성.jpg
이미지 저장 완료: ./img/자보.jpg
이미지 저장 완료: ./img/상조.jpg
이미지 저장 완료: ./img/자려전.jpg
이미지 저장 완료: ./img/입전.jpg
이미지 저장 완료: ./img/일레인.jpg
이미지 저장 완료: ./img/홍미인.jpg
이미지 저장 완료: ./img/성미인.jpg
이미지 저장 완료: ./img/군작.jpg
이미지 저장 완료: ./img/신화월.jpg
이미지 저장 완료: ./img/무을녀.jpg
이미지 저장 완료: ./img/우주목.jpg
이미지 저장 완료: ./img/십자성.jpg
이미지 저장 완료: ./img/남십자성.jpg
이미지 저장 완료: ./img/염자.jpg
이미지 저장 완료: ./img/애성.jpg
이미지 저장 완료: ./img/기천.jpg
이미지 저장 완료: ./img/은전.jpg
이미지 저장 완료: ./img/백로.jpg
이미지 저장 완료: ./img/크라슐라 ‘다비드’.jpg
이미지 저장 완료: ./img/크라슐라 ‘부다템플’.jpg
이미지 저장 완료: ./img/적귀성.jpg
이미지 저장 완료: ./img/화제.jpg
이미지 저장 완료: ./img/복랑.jpg
이미지 저장 완료: ./img/백미인.jpg
이미지 저장 완료: ./img/웅동자.jpg
이미지 저장 완료: ./img/월토이.jpg
이미지 저

## 크롤링 완료한 데이터

In [1]:
import pandas as pd

# 잎을 관상하는 실내식물
leaf_info_df = pd.read_csv("../data/leaf_info.csv")
leaf_detail_df = pd.read_csv("../data/leaf_detail.csv")
leaf_manage_df = pd.read_csv("../data/leaf_manage.csv")

# 건조에 강한 실내식물
dry_info_df = pd.read_csv("../data/dry_info.csv")
dry_manage_df = pd.read_csv("../data/dry_manage.csv")

# 분류
list_df_1 = pd.read_csv("../data/list_1.csv")
list_df_2 = pd.read_csv("../data/list_2.csv")

### 잎을 감상하는 실내식물

In [3]:
leaf_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      216 non-null    object
 1   학명      216 non-null    object
 2   영명      215 non-null    object
 3   유통명     104 non-null    object
 4   과명      215 non-null    object
 5   원산지     205 non-null    object
 6   TIP     82 non-null     object
dtypes: object(7)
memory usage: 11.9+ KB


In [7]:
leaf_info_df.head()

,이름,학명,영명,유통명,과명,원산지,TIP
0,가울테리아,Gaultheria procumbens,"Checkerberry, Eastern Teaberry, (Spreading) Wi...",파스향나무 (추천 유통명: 가울테리아),진달래과,"아시아, 아메리카,캐나다","식용, 지피, 약용, 향기,관엽, 관화, 관실"
1,개운죽,Dracaena sanderiana 'Virens',Sander's dracaena,개운죽,백합과,서부 아프리카,대나무를 닮아서 이름까지 얻게 되었지만 사실은 대나무가 아닌 드라세나속의 관엽
2,골드크레스트 '윌마',Cupressus macrocarpa 'Wilma',Monterey cypress,"율마, 골드크리스터 (추천 유통명: 윌마)",측백나무과,북부 아메리카,부드러운 연두빛의 침엽수를 연상하게 하는 실내식물로 허브 향을 지님
3,공작야자,Caryota mitis,"Fish tail palm, Burmese Fishtail Palm, Cluster...","공작야자, (추천 유통명: 미티스야자)",야자과,"동남아시아, 인도네시아",방풍 / 차폐 / 가장자리 경계
4,관엽베고니아,Begonia spp.,foliage begonia,NaN,베고니아과,"인도, 남아메리카원산식물의 교잡종",NaN


In [8]:
leaf_detail_df.head()

,이름,분류,생육형태,생장높이,생장너비,실내정원구성,생태형,잎형태,잎무늬,잎색,꽃피는 계절,꽃색,열매 맺는 계절,열매색,향기,번식방법,번식시기
0,가울테리아,"잎&꽃보기식물, 열매보기식물",관목형,NaN,NaN,"하층목,지피식물",일반형,"화려함, 잎의 질감-중간, 잎의 광택-있음, 상록",기타 (무늬없음 등),"녹색, 연두색",여름,"분홍색,흰색",NaN,빨강색,중간,"파종,삽목,분주",파종-9~11월/분주-3~5월
1,개운죽,잎보기식물,직립형,NaN,NaN,"중층목,하층목","일반형,수경형",NaN,기타 (무늬없음 등),"녹색, 연두색",NaN,NaN,NaN,NaN,거의 없음,삽목,NaN
2,골드크레스트 '윌마',잎보기식물,직립형,NaN,NaN,"중층목,하층목",일반형,NaN,기타 (무늬없음 등),"녹색, 연두색",NaN,NaN,NaN,NaN,강함,삽목,NaN
3,공작야자,잎보기식물,직립형,NaN,NaN,상층목,일반형,잎의 광택-있음 / 잎의 질감-굵고 선명한 / 상록 / 껍질질감- 부드러움 / 가시-없음,기타 (무늬없음 등),"녹색, 연두색",여름,흰색,NaN,빨강색,NaN,파종,NaN
4,관엽베고니아,잎&꽃보기식물,관목형,NaN,NaN,"하층목,지피식물",일반형,NaN,점무늬,"녹색, 연두색,은색, 회색,빨강, 분홍, 자주색",봄,분홍색,NaN,NaN,NaN,삽목,NaN


In [9]:
leaf_manage_df.head()

,이름,관리수준,관리요구도,광요구도,배치장소,생장속도,생육적온,겨울최저온도,습도,비료,토양,물주기-봄,물주기-여름,물주기-가을,물주기-겨울,병충해
0,가울테리아,경험자,낮음 (잘 견딤),"중간 광도(800~1,500 Lux),높은 광도(1,500~10,000 Lux)",거실 창측 (실내깊이 150~300cm)\n발코니 내측 (실내깊이 50~150cm)...,느림,16~20℃,0℃ 이하,40 ~ 70%,비료를 보통 요구함,"토양 : 중성,산성 / 배수 잘 됨 (Loam,Sand)",토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,화분 흙 대부분 말랐을때 충분히 관수함,"응애,깍지벌레"
1,개운죽,초보자,낮음 (잘 견딤),"낮은 광도(300~800 Lux),중간 광도(800~1,500 Lux),높은 광도(...",거실 내측 (실내깊이 300~500cm)\n거실 창측 (실내깊이 150~300cm)...,보통,16~20℃,13℃ 이상,40 ~ 70%,비료를 거의 요구하지않음,NaN,흙을 촉촉하게 유지함(물에 잠기지 않도록 주의),흙을 촉촉하게 유지함(물에 잠기지 않도록 주의),흙을 촉촉하게 유지함(물에 잠기지 않도록 주의),토양 표면이 말랐을때 충분히 관수함,"응애,깍지벌레"
2,골드크레스트 '윌마',경험자,보통 (약간 잘 견딤),"높은 광도(1,500~10,000 Lux)",거실 창측 (실내깊이 150~300cm)\n발코니 내측 (실내깊이 50~150cm)...,보통,16~20℃,10℃,40 ~ 70%,비료를 보통 요구함,NaN,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,화분 흙 대부분 말랐을때 충분히 관수함,"응애,깍지벌레"
3,공작야자,경험자,필요함,"중간 광도(800~1,500 Lux),높은 광도(1,500~10,000 Lux)",거실 창측 (실내깊이 150~300cm)\n발코니 내측 (실내깊이 50~150cm)...,보통,21~25℃,13℃ 이상,70% 이상,비료를 보통 요구함,"토양-중성,산성/배수잘됨/타입-Loam",토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,"응애,깍지벌레"
4,관엽베고니아,경험자,보통 (약간 잘 견딤),"중간 광도(800~1,500 Lux),높은 광도(1,500~10,000 Lux)",거실 창측 (실내깊이 150~300cm)\n발코니 내측 (실내깊이 50~150cm)...,보통,16~20℃,13℃ 이상,40 ~ 70%,비료를 보통 요구함,NaN,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,토양 표면이 말랐을때 충분히 관수함,"응애,깍지벌레,온실가루이"


### 건조에 강한 실내식물

In [4]:
dry_info_df.head()

,이름,학명,유통명,형태분류,원산지,꽃,엽색변화,뿌리형태,생장형,생장속도,특성,월동온도,생육온도
0,페페로미아 그라베올렌스,"Peperomia graveolens, 후추과",NaN,불규칙형,페루,NaN,"잎의 앞면은 초록, 잎의 뒷면은 붉은색을 띰",가는 뿌리,봄~가을 성장 (겨울에 휴면),느림,NaN,10°C,NaN
1,산세베리아 ‘하니’,"Sansevieria trifasciata 'Hahnii' , 용설란과",NaN,로제트형,아프리카 콩고,한 여름 무렵에 흰색 꽃이 핌,NaN,굵은 뿌리,봄~가을 성장 (겨울에 휴면),빠름,높이 15cm\n잎은 넓고 다양한 무늬가 있음,2~3°C,18~27°C
2,미파,"Faucaria tigrina 'Kikunami' , 석류풀과",경파,로제트형,남아프리카,초가을부터 민들레처럼 생긴 노란색 꽃이 핌\n낮에만 핌,녹색,가는 뿌리,"가을~봄 성장 (겨울생장형, 여름에 휴면)",보통,원예선발품종으로 식물체의 지름이 4cm 정도이며\n잎의 단면은 삼각형 모양을 이루고...,0°C,NaN
3,벽어연,"Echinus maximilianus, 석류풀과",NaN,불규칙형,남아프리카 남부,여름에 줄기 끝에서 노란색으로 한 송이씩 두상화로 달림\n꽃잎의 수가 많으며 지름은...,온도 차가 심해지면 잎 끝부분이 붉게 물듦,가는 뿌리,가을~봄 성장 (겨울생장형),보통,"키가 90cm인 관목형으로 잎은 가위모양이며,\n포복성이 있어 길어지면 옆으로 누움...",0°C,NaN
4,입전봉,"Sinocrassula densirosulata, 돌나물과",NaN,군생형,NaN,NaN,잎 뒷면이 붉은색으로 변함,가는 뿌리,봄~가을 성장,빠름,NaN,5°C,NaN


In [5]:
dry_manage_df.head()

,이름,광,물주기,번식,관리수준,관리요구도,배치장소,병충해,TIP
0,페페로미아 그라베올렌스,충분한 광 필요,분 흙이 마르면 흠뻑 관수 한겨울에는 단수,줄기꽂이,어려움,필요함,발코니 내측까지 이용 가능함\n- 창측의 형태 그대로 유지,NaN,NaN
1,산세베리아 ‘하니’,고온에 강하므로 연중 광선이 잘 쬐는 반그늘\n아래에서 잘 자라고 여름철에는 직사광...,분 흙이 충분히 마른 뒤 며칠 후 충분히 관수,"포기나누기, 잎꽂이",쉬움,약간 돌봄,발코니 내측까지 이용 가능함\n- 형태와 색상 모두 유지하고 싱싱한 녹색 빛을 띰,병충해에는 별로 걸리지 않으나 잎을 깨끗이 닦아주도록 함,NaN
2,미파,"반그늘~부분적인 강광, 실내 밝은 곳에서 키움\n한 여름에는 반그늘에서 키움",한 달에 한 번 정도 과습하지 않게 관리\n여름철에는 물을 거의 주지 않음,잎꽂이,어려움,필요함,발코니 내측까지 이용 가능함\n- 창측과 형태와 색상 모두 유지함,NaN,NaN
3,벽어연,반그늘이나 부분적인 강광,잎에 주름이 생기면 관수\n휴면기인 여름철에는 단수하고 서늘한 곳에 둠,"자구꽂이, 포기나누기",보통,약간 돌봄,발코니 내측까지 이용 가능함\n- 창측과 형태와 색상 모두 유지함,NaN,NaN
4,입전봉,충분한 광 필요,여름철에는 물을 적게 줌,"잎꽂이, 줄기꽂이",매우 어려움,필요함,발코니 내측까지 이용 가능함\n- 약간 부피감이 커지고 붉은색이 연해짐,NaN,NaN


## 분류

In [6]:
list_df_1.head()

,name,class
0,가울테리아,leaf
1,개운죽,leaf
2,골드크레스트 '윌마',leaf
3,공작야자,leaf
4,관엽베고니아,leaf


In [17]:
list_df_2.head()

,name,class
0,페페로미아 그라베올렌스,dry
1,산세베리아 ‘하니’,dry
2,미파,dry
3,벽어연,dry
4,입전봉,dry


In [19]:
list_df = pd.concat([list_df_1, list_df_2])
list_df.head()

,name,class
0,가울테리아,leaf
1,개운죽,leaf
2,골드크레스트 '윌마',leaf
3,공작야자,leaf
4,관엽베고니아,leaf


In [31]:
list_df[list_df.duplicated('name')]

,name,class
167,크로톤,leaf
85,녹영,dry


In [33]:
list_df[list_df['name'] == '크로톤']

,name,class
166,크로톤,leaf
167,크로톤,leaf


In [34]:
list_df[list_df['name'] == '녹영']

,name,class
22,녹영,leaf
85,녹영,dry


In [36]:
list_df.to_csv("list.csv", index=False, encoding='utf-8-sig')